In [1]:
import numpy as np
np.set_printoptions(precision=1)
import tensorflow as tf
import matplotlib.pylab as plt

from modules.utils import load_cifar10
from modules.cnn_with_spectral_parameterization import CNN_Spectral_Param
from modules.cnn_with_spectral_pooling import CNN_Spectral_Pool

% matplotlib inline
% load_ext autoreload
% autoreload 2

In [2]:
xtrain, ytrain, xtest, ytest = load_cifar10(5, channels_last=True)

file already downloaded..
getting batch 1
getting batch 2
getting batch 3
getting batch 4
getting batch 5


In [3]:
xtrain.shape, ytrain.shape, xtest.shape, ytest.shape

((50000, 32, 32, 3), (50000,), (10000, 32, 32, 3), (10000,))

In [8]:
# take a small batch for testing architecture
sampleind = np.random.choice(np.arange(xtrain.shape[0]),
                             size=128, replace=False)
xtrain_sample = xtrain[sampleind]
ytrain_sample = ytrain[sampleind]
assert len(np.unique(ytrain_sample)) == 10
print(np.bincount(ytrain_sample))

[16 19 14  4 17 18  8  8 13 11]


In [14]:
def write_loss(name, losses):
    with open('loss_vals/' + name + '.txt', 'w') as f:
        for loss in losses:
            f.write("%s\n" % loss)

In [26]:
tf.reset_default_graph()
generic_3_spectral = CNN_Spectral_Param(num_output=10, kernel_size=3, architecture='generic', use_spectral_params=True)
generic_3_spectral.train(xtrain, ytrain, None, None,
          batch_size=128, epochs=150)

write_loss('generic_3_spectral', generic_3_spectral.loss_vals)

number of batches for training: 390
epoch 1 
0.6328125
1.85073
epoch 2 
0.5234375
1.91718
epoch 3 
0.671875
1.39846
epoch 4 
0.5625
1.72858
epoch 5 
0.671875
1.36191
epoch 6 
0.640625
1.46903
epoch 7 
0.671875
1.44973
epoch 8 
0.7265625
1.14981
epoch 9 
0.6953125
1.27554
epoch 10 
0.734375
1.2085
epoch 11 
0.7578125
1.14471
epoch 12 
0.703125
1.24633
epoch 13 
0.8203125
1.00689
epoch 14 
0.796875
1.02103
epoch 15 
0.7578125
1.06867
epoch 16 
0.7890625
0.99392
epoch 17 
0.7734375
1.03495
epoch 18 
0.75
1.01953
epoch 19 
0.8125
0.925105
epoch 20 
0.875
0.846347
epoch 21 
0.828125
0.872914
epoch 22 
0.7421875
1.08419
epoch 23 
0.8671875
0.752278
epoch 24 
0.7890625
0.979568
epoch 25 
0.8984375
0.74918
epoch 26 
0.890625
0.801322
epoch 27 
0.84375
0.860379
epoch 28 
0.84375
0.786697
epoch 29 
0.796875
0.958144
epoch 30 
0.84375
0.838189
epoch 31 
0.90625
0.71223
epoch 32 
0.859375
0.761625
epoch 33 
0.8203125
0.961655
epoch 34 
0.8984375
0.749637
epoch 35 
0.921875
0.620633
epoch 36 
0.890

In [ ]:
tf.reset_default_graph()
generic_3_spatial = CNN_Spectral_Param(num_output=10, kernel_size=3, architecture='generic', use_spectral_params=False)
generic_3_spatial.train(xtrain, ytrain, None, None,
          batch_size=128, epochs=150)

write_loss('generic_3_spatial', generic_3_spatial.loss_vals)

number of batches for training: 390
epoch 1 
0.6015625
1.77516
epoch 2 
0.53125
1.77439
epoch 3 
0.703125
1.37846
epoch 4 
0.640625
1.57015
epoch 5 
0.65625
1.37885
epoch 6 
0.6640625
1.44676
epoch 7 
0.75
1.1664
epoch 8 
0.7265625
1.3443
epoch 9 
0.71875
1.25024
epoch 10 
0.75
1.28717
epoch 11 
0.7265625
1.25799
epoch 12 
0.6796875
1.2502
epoch 13 
0.71875
1.23255
epoch 14 
0.640625
1.38184
epoch 15 
0.765625
1.08288
epoch 16 
0.8125
1.04226
epoch 17 
0.7734375
1.02235
epoch 18 
0.7734375
1.09092
epoch 19 
0.8203125
0.977227
epoch 20 
0.7578125
1.09936
epoch 21 
0.78125
0.966377
epoch 22 
0.734375
1.08794
epoch 23 
0.875
0.816486
epoch 24 
0.828125
0.841757
epoch 25 
0.859375
0.813641
epoch 26 
0.8515625
0.843999
epoch 27 
0.84375
0.831119
epoch 28 
0.78125
1.05302
epoch 29 
0.8828125
0.754002
epoch 30 
0.8828125
0.779867
epoch 31 
0.8125
0.854833
epoch 32 
0.8359375
0.728406
epoch 33 
0.859375
0.779914
epoch 34 
0.8203125
0.894122
epoch 35 
0.859375
0.714636
epoch 36 
0.8828125
0.772

In [ ]:
tf.reset_default_graph()
generic_5_spectral = CNN_Spectral_Param(num_output=10, kernel_size=5, architecture='generic', use_spectral_params=True)
generic_5_spectral.train(xtrain, ytrain, None, None,
          batch_size=128, epochs=150)

write_loss('generic_5_spectral', generic_5_spectral.loss_vals)

number of batches for training: 390
epoch 1 
0.546875
2.02192
epoch 2 
0.5
1.93132
epoch 3 
0.6796875
1.71598
epoch 4 
0.6171875
1.68152
epoch 5 
0.6015625
1.58319
epoch 6 
0.578125
1.47588
epoch 7 
0.6953125
1.35494
epoch 8 
0.75
1.17946
epoch 9 
0.65625
1.35685
epoch 10 
0.609375
1.58849
epoch 11 
0.6953125
1.37285
epoch 12 
0.7265625
1.26167
epoch 13 
0.734375
1.17859
epoch 14 
0.765625
1.12891
epoch 15 
0.7890625
0.984723
epoch 16 
0.6953125
1.18813
epoch 17 
0.75
1.04183
epoch 18 
0.7109375
1.17005
epoch 19 
0.7734375
1.08334
epoch 20 
0.7421875
1.1044
epoch 21 
0.7265625
1.02349
epoch 22 
0.7734375
0.958988
epoch 23 
0.7890625
0.965258
epoch 24 
0.765625
1.10233
epoch 25 
0.8359375
0.809763
epoch 26 
0.7421875
1.09003
epoch 27 
0.8203125
0.921457
epoch 28 
0.7734375
0.978346
epoch 29 
0.8515625
0.757812
epoch 30 
0.8359375
0.818167
epoch 31 
0.78125
1.03378
epoch 32 
0.828125
0.899985
epoch 33 
0.84375
0.897321
epoch 34 
0.765625
1.03395
epoch 35 
0.8203125
0.85675
epoch 36 
0.86

In [ ]:
tf.reset_default_graph()
generic_5_spatial = CNN_Spectral_Param(num_output=10, kernel_size=5, architecture='generic', use_spectral_params=False)
generic_5_spatial.train(xtrain_sample, ytrain_sample, None, None,
          batch_size=128, epochs=150)

write_loss('generic_5_spatial', generic_5_spatial.loss_vals)

In [ ]:
xtrain_NCHW, ytrain_NCHW, xtest_NCHW, ytest_NCHW = load_cifar10(5, channels_last=False)

sampleind_NCHW = np.random.choice(np.arange(xtrain_NCHW.shape[0]),
                             size=64, replace=False)
xtrain_sample_NCHW = xtrain_NCHW[sampleind_NCHW]
ytrain_sample_NCHW = ytrain_NCHW[sampleind_NCHW]
assert len(np.unique(ytrain_sample_NCHW)) == 10
print(np.bincount(ytrain_sample_NCHW))

tf.reset_default_graph()
cnn3 = CNN_Spectral_Pool()

cnn3.train(xtrain_sample_NCHW,
          ytrain_sample_NCHW,
          xtrain_sample_NCHW,
          ytrain_sample_NCHW,
          batch_size=64,
          epochs=10,
          extra_conv_layer=True,
          use_global_averaging=True)